# Imports

In [2]:
import logging
import time

import numpy as np
import pandas as pd
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from brainflow import *
from keras import layers
from matplotlib import pyplot as plt
from scipy import signal
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

2023-03-03 12:32:20.296019: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 12:32:20.414661: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-03 12:32:20.414678: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-03 12:32:21.070459: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# Load Data

In [14]:
data_0 = pd.read_csv("../data/60s-l-0.csv")
data_1 = pd.read_csv("../data/60s-arm-flap.csv")

SCALE_FACTOR = (4500000) / 24 / (2**23 - 1)
fs = 250  # 250 samples per second

# trim the first and last 5 seconds off the data
data_0 = data_0.iloc[5 * fs : -5 * fs, :]
data_1 = data_1.iloc[5 * fs : -5 * fs, :]

# scale the eeg columns (eeg1 - eeg8) by SCALE_FACTOR
eeg_cols = [f"eeg{i}" for i in range(1, 9)]
data_0[eeg_cols] = data_0[eeg_cols] * SCALE_FACTOR
data_1[eeg_cols] = data_1[eeg_cols] * SCALE_FACTOR

data = pd.concat([data_0, data_1])
data.head()

,sample,packet,eeg1,eeg2,eeg3,eeg4,eeg5,eeg6,eeg7,eeg8,...,other3,other4,other5,other6,other7,analog1,analog2,analog3,timestamp,marker
1250,1250,226.0,479.456089,808.791727,-999.518706,-890.343511,3976.856293,162.700392,-975.344538,-400.449769,...,80.0,30.0,128.0,2.0,64.0,0.0,0.0,0.0,1.676478e+09,0
1251,1251,227.0,480.861965,809.349281,-997.780097,-889.953323,3978.641865,163.546715,-967.924472,-394.095350,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.676478e+09,0
1252,1252,228.0,482.959787,811.613970,-997.053678,-890.076225,3979.265367,163.935904,-961.269294,-386.629321,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.676478e+09,0
1253,1253,229.0,481.597876,811.125360,-999.174981,-890.885578,3977.095102,162.627950,-968.572453,-393.160098,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.676478e+09,0
1254,1254,230.0,479.488563,808.964089,-999.854937,-890.640274,3976.815326,162.475572,-975.514402,-400.742535,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.676478e+09,0


## Scale and trim data

In [ ]:
SCALE_FACTOR = (4500000) / 24 / (2**23 - 1)

In [ ]:
# Using 250 samples per second
fs = 250

# trim and scale